In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tqdm import tqdm

In [2]:
EPOCHS = 100

In [3]:
# List all available devices detected by TensorFlow
print("Available devices:")
devices = tf.config.list_physical_devices()
for device in devices:
    print(device)

Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


## Load and prepare data

In [4]:
# Load the data
data = pd.read_csv("../dataset.csv")

In [5]:
# Separate features and target
X = data.drop("Label", axis=1)
y = data["Label"]

In [6]:
# Convert categorical data to numeric if necessary
X = pd.get_dummies(X, sparse=True)

In [7]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:1059: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  if np.may_share_memory(array, array_orig):
/Users/odai/cs5525-research-paper-submission/.venv/lib/python3.11/site-packages/sklearn/utils/validation.py:869: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy arr

## Build an AutoEncoder

In [9]:
# Define the encoder
input_layer = Input(shape=(X_train_scaled.shape[1],))
encoded = Dense(128, activation="relu")(input_layer)
encoded = Dense(64, activation="relu")(encoded)

In [10]:
# Define the decoder
decoded = Dense(128, activation="relu")(encoded)
decoded = Dense(X_train_scaled.shape[1], activation="sigmoid")(decoded)

In [11]:
# Define the autoencoder
autoencoder = Model(input_layer, decoded)

# Encoder for predictions
encoder = Model(input_layer, encoded)

# Prediction model
predictor = Dense(1, activation="sigmoid")(encoded)
prediction_model = Model(input_layer, predictor)

# Compile models
autoencoder.compile(optimizer="adam", loss="mse")
prediction_model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [12]:
# Display the model summary
autoencoder.summary()
prediction_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 20090)          │     2,591,610 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,179,834 (19.76 MB)

 Trainable params: 5,179,834 (19.76 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 20090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,571,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,579,969 (9.84 MB)

 Trainable params: 2,579,969 (9.84 MB)

 Non-trainable params: 0 (0.00 B)

## Train and evaluate

In [13]:
# Train the autoencoder
autoencoder.fit(
    X_train_scaled, X_train_scaled, epochs=EPOCHS, batch_size=256, validation_split=0.2
)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 68ms/step - loss: 0.9828 - val_loss: 0.8041
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.8009 - val_loss: 0.8013
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - loss: 0.8008 - val_loss: 0.8012
Epoch 12/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step

In [14]:
# Train the prediction model
prediction_model.fit(
    X_train_scaled, y_train, epochs=EPOCHS, batch_size=256, validation_split=0.2
)

Epoch 1/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.5126 - loss: 6.4110 - val_accuracy: 0.9281 - val_loss: 0.3764
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.8942 - loss: 0.2887 - val_accuracy: 0.9887 - val_loss: 0.2082
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.9965 - loss: 0.0131 - val_accuracy: 0.9931 - val_loss: 0.1760
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.9997 - loss: 0.0021 - val_accuracy: 0.9950 - val_loss: 0.1709
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9998 - loss: 0.0012 - val_accuracy: 0.9950 - val_loss: 0.1695
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 1.0000 - loss: 7.5879e-04 - val_accuracy: 0.9950 - val_loss: 0.1686
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 1.0000 - loss: 6.1408e-04 - val_accuracy: 0.9950 - val_loss: 0.1677
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 1.0000 - loss: 5.1768e-04 - val_

In [15]:
# Evaluate the prediction model
test_loss, test_acc = prediction_model.evaluate(X_test_scaled, y_test)
print("Test Accuracy: {:.2f}%".format(test_acc * 100))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4809 - loss: 5.7882
Test Accuracy: 49.50%


In [18]:
# Pickle the models
autoencoder.save("checkpoints/ae.h5")
prediction_model.save("checkpoints/ae_prediction.h5")